In [29]:
from google.cloud import storage, bigquery
import pandas as pd
from google.oauth2 import service_account
import json

# Set up Google Cloud credentials
project_credentials = service_account.Credentials.from_service_account_file('data-finance-final-92d8049c252f.json')
project_id = 'data-finance-final'

# Initialize the client
storage_client = storage.Client(credentials=project_credentials, project=project_id)
bucket_name = 'data_finance_final'

# Get the bucket
bucket = storage_client.get_bucket(bucket_name)

# List all blobs in the bucket
blobs = list(bucket.list_blobs())

# Process the blob names to find the most recent file for each stock
latest_files = {}
for blob in blobs:
    # Assuming file names are in 'Price_STOCK-DATE.json' format
    parts = blob.name.split('_')
    #print(parts)
    if len(parts) > 1 and parts[0] == 'price/Price':
        # Extract the stock symbol and date
        stock, date_str = parts[1], parts[2].split('.')[0]
        date = pd.to_datetime(date_str, format='%Y-%m-%d')
        
        # Determine if this file is the most recent for the stock
        if stock not in latest_files or date > latest_files[stock]['date']:
            latest_files[stock] = {'date': date, 'blob': blob}

# Initialize BigQuery client
bigquery_client = bigquery.Client(credentials=project_credentials, project=project_id)
dataset_id = 'stockMetaData'
table_id = 'stock_prices_cleaned'

# Define the schema of your BigQuery table
schema = [
    bigquery.SchemaField('date', 'DATE'),
    bigquery.SchemaField('open', 'FLOAT'),
    bigquery.SchemaField('high', 'FLOAT'),
    bigquery.SchemaField('low', 'FLOAT'),
    bigquery.SchemaField('close', 'FLOAT'),
    bigquery.SchemaField('volume', 'INTEGER'),
    bigquery.SchemaField('stock', 'String')

]

# Create or get the dataset and table
dataset_ref = bigquery_client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
bigquery_client.create_dataset(dataset, exists_ok=True)
table_ref = dataset_ref.table(table_id)
table = bigquery.Table(table_ref, schema=schema)
bigquery_client.create_table(table, exists_ok=True)

# For each stock, load the most recent data into BigQuery
for stock, file_info in latest_files.items():
    print(stock)
    
    try:
        # Download the blob to a local variable
        data_string = file_info['blob'].download_as_string()
        data_json = json.loads(data_string)

        # Process and clean the data as needed, then convert to DataFrame
        # This is an example, you'll need to adjust it to match your JSON structure
        df = pd.DataFrame.from_dict(data_json['Time Series (Daily)'], orient='index')
        df.index = pd.to_datetime(df.index)
        df.sort_index(inplace=True)

        # Reset the index and rename columns
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'date'}, inplace=True)
        df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
        df['stock'] = stock

        df['open'] = pd.to_numeric(df['open'], errors='coerce')
        df['high'] = pd.to_numeric(df['high'], errors='coerce')
        df['low'] = pd.to_numeric(df['low'], errors='coerce')
        df['close'] = pd.to_numeric(df['close'], errors='coerce')
        df['volume'] = pd.to_numeric(df['volume'], downcast='integer', errors='coerce')
        df['date'] = pd.to_datetime(df['date'])
        print(df)
        # If the table doesn't exist, create it, otherwise append the data
        df.to_gbq(destination_table=f'{dataset_id}.{table_id}', project_id=project_id, if_exists='append', credentials=project_credentials)
    except KeyError as e:
        print(e)


AAPL
         date    open    high       low   close    volume stock
0  2023-07-20  195.09  196.47  192.4950  193.13  59581196  AAPL
1  2023-07-21  194.10  194.97  191.2300  191.94  71951683  AAPL
2  2023-07-24  193.41  194.91  192.2500  192.75  45505097  AAPL
3  2023-07-25  193.33  194.44  192.9150  193.62  37283201  AAPL
4  2023-07-26  193.67  195.64  193.3200  194.50  47471868  AAPL
..        ...     ...     ...       ...     ...       ...   ...
95 2023-12-04  189.98  190.05  187.4511  189.43  43389519  AAPL
96 2023-12-05  190.21  194.40  190.1800  193.42  66628398  AAPL
97 2023-12-06  194.45  194.76  192.1100  192.32  40895115  AAPL
98 2023-12-07  193.63  195.00  193.5900  194.27  47477655  AAPL
99 2023-12-08  194.20  195.99  193.6700  195.71  53406358  AAPL

[100 rows x 7 columns]


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


SPX
'Time Series (Daily)'
V
         date     open     high     low   close   volume stock
0  2023-07-20  241.160  241.610  239.07  239.62  4694950     V
1  2023-07-21  239.740  240.345  238.73  239.25  5427353     V
2  2023-07-24  239.680  241.275  238.19  240.74  6009950     V
3  2023-07-25  240.020  240.800  238.56  238.69  4997534     V
4  2023-07-26  231.415  238.850  227.68  237.10  7551396     V
..        ...      ...      ...     ...     ...      ...   ...
95 2023-12-04  255.910  257.385  254.35  254.44  5247450     V
96 2023-12-05  254.190  254.820  252.14  254.61  4199094     V
97 2023-12-06  255.800  256.830  253.53  254.29  3804631     V
98 2023-12-07  254.890  256.140  253.50  255.82  3589256     V
99 2023-12-08  255.000  256.040  253.87  255.74  3732515     V

[100 rows x 7 columns]


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
